In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = './data/stage1_train/'
TEST_PATH = './data/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed
test_ids = next(os.walk(TEST_PATH))[1]

/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

Getting and resizing train images and masks ... 
Getting and resizing test images ... 


100%|██████████| 65/65 [00:01<00:00, 60.56it/s]

Done!


In [3]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [8]:
# Predict on train, val and test
model = load_model('./checkpoint', custom_objects={'mean_iou': mean_iou})
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_test_t = (preds_test > 0.8).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

65/65 [==============================] - 84s 1s/step


In [9]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [10]:
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

In [11]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018-1.csv', index=False)

In [63]:
y_true = np.array([[1,1],[1,1]0])
y_pred = np.array([[1,1],[1,0]])
y_true = tf.to_int32(y_true)
y_pred_ = tf.to_int32(y_pred)

In [66]:
score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 1)

In [67]:
sess = tf.Session()
sess.run(score)

FailedPreconditionError: Attempting to use uninitialized value mean_iou_11/total_confusion_matrix
	 [[Node: mean_iou_11/total_confusion_matrix/read = Identity[T=DT_DOUBLE, _class=["loc:@mean_iou_11/total_confusion_matrix"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](mean_iou_11/total_confusion_matrix)]]

Caused by op 'mean_iou_11/total_confusion_matrix/read', defined at:
  File "/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/afs/inf.ed.ac.uk/user/s17/s1777953/miniconda3/envs/mlp/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    

In [57]:
up_opt

<tf.Tensor 'mean_iou_7/AssignAdd:0' shape=(2, 2) dtype=float64_ref>

In [59]:
tf.metrics.mean_iou(y_true, y_pred_, 2)

(<tf.Tensor 'mean_iou_9/mean_iou:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_iou_9/AssignAdd:0' shape=(2, 2) dtype=float64_ref>)